In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize
import torch.nn as nn
from torch.optim import Adam
import gc

#constants
grayscale = True
height = 256
width = 256
train_data = {'id':[],'label':[]}

#read input file
with open('/content/gdrive/My Drive/Colab Notebooks/pressAndpic.txt','r') as f:
    t = f.readlines()

#split data to feature-label
for i in t:
    a,b = i.split("  ")
    lb = b.replace("\n","")
    train_data['id'].append(a)
    train_data['label'].append(lb)

#preprocess the image
def preprocess(img):
    img_size = (height, width)
    img = resize(np.array(img), img_size)
    img = np.transpose(img)
    img = img.astype('float32')
    return img

train_img = []
for img_path in tqdm(train_data['id']):
    img = imread(img_path, as_gray=True)
    train_img.append(preprocess(img))

100%|██████████| 3010/3010 [25:04<00:00,  2.00it/s]


In [8]:
#defining the train data
train_x = np.array(train_img)
# defining the train label
train_y = np.array(train_data['label'])

#divide into train and validation set
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.1)

#set number of epochs
n_epochs = 50   #44 normal and #5 is lab test
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []

# converting the target into torch format
val_x = val_x.reshape(val_x.shape[0], 1,val_x.shape[1],val_x.shape[2])
val_x  = torch.from_numpy(val_x)

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)

x_val, y_val = val_x,val_y
y_val = y_val.type(torch.LongTensor)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cnn_layers = nn.Sequential(
        # Defining a 2D convolution layer
        nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        # Defining another 2D convolution layer
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2),
        nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
        nn.BatchNorm2d(4),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(4096,1024),
            nn.Linear(1024,512),
            nn.Linear(512,2),
        )

    # Defining the forward pass
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [9]:
model = Net()
# model.load_state_dict(torch.load('/content/gdrive/My Drive/Colab Notebooks/ml-for-games-final/Transfer Learning/ss_weights_aug.pt'))
# model.eval()

optimizer = Adam(model.parameters(), lr=3e-4, weight_decay= 1e-5)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

In [10]:
def train(epoch, new_x_train, new_y_train):
    model.train()
    tr_loss = 0
    # converting the data into GPU format
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()

    # prediction for training and validation set
    output_train = model(new_x_train)

    running_corrects_train = 0
    #computing the accuracy
    _, preds_train = torch.max(output_train, 1)
    running_corrects_train += torch.sum(preds_train == new_y_train)

    # computing the training and validation loss
    loss_train = criterion(output_train, new_y_train)
    train_losses.append(loss_train.item())

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()

    model.eval()
    output_val = model(x_val)
    running_corrects_val = 0
    _, preds_test = torch.max(output_val, 1)
    running_corrects_val += torch.sum(preds_test == y_val)
    loss_val = criterion(output_val, y_val)
    val_loss = loss_val.item()
    val_losses.append(val_loss)

    # Metrics
    if iteration == 31: # Last Iteration
      correct_true = torch.sum(preds_test * y_val == 1).float()
      target_true = torch.sum(y_val == 1).float()
      predict_true = torch.sum(preds_test == 1).float()
      precision = correct_true / predict_true
      recall = correct_true / target_true
      f1_score = 2 * precision * recall / (precision + recall)

      global precision_list, recall_list, f1_score_list, val_loss_list, val_accuracy_list
      precision_list += [precision.item()]
      recall_list += [recall.item()]
      f1_score_list += [f1_score.item()]

      val_accuracy = running_corrects_val / len(y_val)

      val_loss_list += [val_loss]
      val_accuracy_list += [val_accuracy.item()]

    if epoch % 2 == 0:
        # printing the validation loss
        print('Epoch : ', epoch + 1, '\t', 'loss :', loss_val, 'train_acc :', running_corrects_train.double(), 'val_acc :', running_corrects_val.double())

x_val = x_val.to('cuda:0')
y_val = y_val.to('cuda:0')

min_error = float('+inf')
minima_flag = 0
best_model = model
tolerance = 20

epoch_list, val_loss_list, iteration_error_list, val_accuracy_list = [], [], [], []
precision_list, recall_list, f1_score_list = [], [], []

# training the model
for epoch in range(n_epochs):
  val_losses = []
  epoch_list += [epoch]
  for iteration in range(32):
    indices = np.random.choice(len(train_x),64)
    ipx = torch.from_numpy(train_x[indices].astype(np.float32)).to('cuda:0')
    ipx2 = torch.flip(ipx,(2,))
    ipy = torch.from_numpy(train_y[indices].astype(np.long)).to('cuda:0')
    ipy2 = 1 - ipy
    ipx = torch.cat((ipx,ipx2),dim = 0)
    ipy = torch.cat((ipy,ipy2),dim = 0)
    ipx = ipx.view(ipx.shape[0],1,ipx.shape[1],ipx.shape[2])
    train(epoch,ipx,ipy)
    del ipx
    del ipy
    # gc.collect()
    # torch.cuda.empty_cache()
  mean_iteration_error = np.mean(val_losses)

  mean_iteration_error = np.mean(val_losses)
  iteration_error_list += [mean_iteration_error]

  if mean_iteration_error < min_error:
    min_error = mean_iteration_error
    best_model = model.cpu().state_dict()
    model.cuda()
    #torch.save(model.cpu().state_dict(), PATH)
    minima_flag = 0
  else:
    minima_flag += 1
    if minima_flag > tolerance:
      break

print(str(min_error))

PATH = '/content/gdrive/My Drive/Colab Notebooks/ml-for-games-final/Transfer Learning/imit_model_aug.pt'
torch.save(best_model, PATH)

Epoch :  1 	 loss : tensor(0.6960, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(67., device='cuda:0', dtype=torch.float64) val_acc : tensor(163., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.7203, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(64., device='cuda:0', dtype=torch.float64) val_acc : tensor(138., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.7126, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(64., device='cuda:0', dtype=torch.float64) val_acc : tensor(138., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6898, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(67., device='cuda:0', dtype=torch.float64) val_acc : tensor(163., device='cuda:0', dtype=torch.float64)
Epoch :  1 	 loss : tensor(0.6913, device='cuda:0', grad_fn=<NllLossBackward>) train_acc : tensor(73., device='cuda:0', dtype=torch.float64) val_acc : tensor(163., device='cuda:0', dtype=torch

In [11]:
print(epoch_list)
print(val_loss_list)
print(iteration_error_list)
print(val_accuracy_list)
print(precision_list)
print(recall_list)
print(f1_score_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
[0.6185685396194458, 0.21181310713291168, 0.17551006376743317, 0.15773352980613708, 0.19265274703502655, 0.13451555371284485, 0.1368807852268219, 0.1536148339509964, 0.14197729527950287, 0.15318603813648224, 0.18590432405471802, 0.20468886196613312, 0.16822797060012817, 0.18970397114753723, 0.17430299520492554, 0.18357621133327484, 0.259812593460083, 0.17524249851703644, 0.36315202713012695, 0.1909257471561432, 0.21716904640197754, 0.2630397379398346, 0.34799525141716003, 0.20361363887786865, 0.17493747174739838, 0.2235487848520279, 0.2236134260892868]
[0.6880691964179277, 0.3804152561351657, 0.17985640885308385, 0.1585849211551249, 0.15902113169431686, 0.15517320716753602, 0.16056001791730523, 0.18046111706644297, 0.1652519847266376, 0.17436014162376523, 0.18405502336099744, 0.18703422928228974, 0.19608061388134956, 0.20753969345241785, 0.2650118903256953, 0.20674634585157037, 0.21954764